In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import csv

In [2]:
data = pd.read_csv('/content/drive/MyDrive/hakaton_dano/data_hackathon_28062024.csv', delimiter=';', on_bad_lines='skip')


In [3]:
def remove_outliers(df, column):
    percentile25 = df[column].quantile(0.25)
    percentile75 = df[column].quantile(0.75)
    iqr = percentile75-percentile25
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr
    df = df[(df[column] < upper_limit) & (df[column] > lower_limit)]
    return df

data = remove_outliers(data, "finish_time")


In [4]:
data["rain.1h"] = data["rain.1h"].fillna("0")
data["snow.1h"] = data["snow.1h"].fillna("0")

<ipython-input-4-8af9a52c3c2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["rain.1h"] = data["rain.1h"].fillna("0")
<ipython-input-4-8af9a52c3c2a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["snow.1h"] = data["snow.1h"].fillna("0")


In [5]:
data = data[data["position"] <= data["n_runners"]]
data = data[data['athlete_age'] < 100]

['record_id',
 'athlete_id',
 'event_name',
 'event_date',
 'n_runners',
 'finish_time',
 'position',
 'athlete_gender',
 'athlete_age',
 'best_time',
 'worst_time',
 'mean_time',
 'median_time',
 'times',
 'num_locations',
 'home',
 'temp',
 'feels_like',
 'pressure',
 'humidity',
 'dew_point',
 'clouds',
 'wind_speed',
 'wind_deg',
 'weather.main',
 'weather.description',
 'rain.1h',
 'snow.1h',
 'track_description_surface']

In [6]:
names = ["feels_like", "wind_speed",
"mean_time", "temp", "dew_point", "rain.1h", "snow.1h"]

for st in names:
  data[st] = data[st].str.replace(',','.')
  data[st] = data[st].astype(float)


In [12]:
events = list(set(data["record_id"]))
di = dict()
di["record_id"] = events
columns = ['event_date', 'n_runners', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'weather.main', 'weather.description', 'rain.1h',
           'snow.1h', 'track_description_surface']
for st in columns:
  c = []
  for line in events:
    c.append(data[data["record_id"] == line][st].iloc[0])
  di[st] = c

ev = pd.DataFrame(di)


In [20]:
ev

,record_id,event_date,n_runners,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface
0,4752,1.1.2023,18,-7.15,-12.41,1029,91,-8.23,100,3.30,161,Snow,light snow,0.0,0.19,асфальт
1,4753,1.1.2023,22,-28.01,-28.01,1028,75,-30.79,75,1.00,120,Snow,light snow,0.0,0.00,смешанное
2,4755,1.1.2023,10,-30.18,-30.18,1034,75,-32.91,79,1.29,30,Clouds,broken clouds,0.0,0.00,твердое
3,4758,1.1.2023,28,1.24,-3.48,1009,98,0.96,100,5.22,211,Snow,light snow,0.0,0.22,твердое
4,4762,1.1.2023,29,2.96,-2.32,996,88,1.17,100,7.56,263,Clouds,overcast clouds,0.0,0.00,грунт
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5302,14130,30.12.2023,45,-16.65,-23.65,1000,84,-18.50,20,5.00,240,Clouds,few clouds,0.0,0.00,твердое
5303,14263,30.12.2023,96,-1.41,-3.65,1021,74,-4.98,0,1.68,157,Clear,clear sky,0.0,0.00,твердое
5304,14274,5.8.2023,29,25.86,26.20,1011,65,18.78,0,0.00,0,Clear,clear sky,0.0,0.00,беговая дорожка
5305,15198,30.12.2023,17,1.63,-3.07,997,95,0.92,100,5.36,265,Snow,light snow,0.0,0.18,твердое


In [18]:
vibros = ['n_runners', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point',
          'wind_speed', 'rain.1h', 'snow.1h']
for elem in vibros:
  ev = remove_outliers(ev, elem)

In [22]:
ev.head()

,record_id,event_date,n_runners,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface
0,4752,1.1.2023,18,-7.15,-12.41,1029,91,-8.23,100,3.30,161,Snow,light snow,0.0,0.19,асфальт
1,4753,1.1.2023,22,-28.01,-28.01,1028,75,-30.79,75,1.00,120,Snow,light snow,0.0,0.00,смешанное
2,4755,1.1.2023,10,-30.18,-30.18,1034,75,-32.91,79,1.29,30,Clouds,broken clouds,0.0,0.00,твердое
3,4758,1.1.2023,28,1.24,-3.48,1009,98,0.96,100,5.22,211,Snow,light snow,0.0,0.22,твердое
4,4762,1.1.2023,29,2.96,-2.32,996,88,1.17,100,7.56,263,Clouds,overcast clouds,0.0,0.00,грунт


In [14]:
data = data[data["record_id"].isin(list(ev["record_id"]))]

In [26]:
data.to_csv("chisteyshaya_data.csv")

In [16]:
ev[ev["weather.main"] == "Rain"]

,record_id,event_date,n_runners,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface
9,4783,1.1.2023,32,3.86,-1.31,993,88,2.06,100,8.00,260,Rain,light rain,0.25,0.00,смешанное
23,4819,1.1.2023,11,3.91,-1.24,993,88,2.11,100,8.00,260,Rain,light rain,0.25,0.00,твердое
76,4985,14.1.2023,37,1.08,-3.57,1004,95,0.37,100,5.00,160,Rain,light intensity shower rain,0.25,0.00,смешанное
108,5050,14.1.2023,35,1.16,-3.47,1007,95,0.45,100,5.00,160,Rain,light intensity shower rain,0.00,0.22,смешанное
111,5055,14.1.2023,18,1.56,-2.37,1010,98,1.28,100,4.00,170,Rain,light rain,0.00,0.00,твердое
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5249,13813,30.12.2023,71,3.31,-0.23,995,100,3.31,75,4.02,180,Rain,light rain,0.21,0.00,твердое
5250,13814,30.12.2023,27,1.91,-3.84,997,93,0.90,100,8.00,260,Rain,light rain,0.00,0.00,твердое
5256,13828,30.12.2023,22,2.84,-0.81,997,100,2.84,100,4.02,260,Rain,light rain,0.11,0.00,твердое
5268,13845,30.12.2023,36,5.63,1.99,1002,87,3.64,99,5.29,244,Rain,light rain,0.13,0.00,"асфальт, бетон"


In [17]:
ev[((ev["weather.main"] == "Snow") & (ev["snow.1h"] == 0))]

,record_id,event_date,n_runners,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface
1,4753,1.1.2023,22,-28.01,-28.01,1028,75,-30.79,75,1.00,120,Snow,light snow,0.0,0.0,смешанное
5,4768,1.1.2023,19,-25.97,-32.71,1028,83,-27.81,40,2.00,130,Snow,light snow,0.0,0.0,твердое
22,4817,1.1.2023,21,-28.01,-28.01,1028,75,-30.79,75,1.00,120,Snow,light snow,0.0,0.0,твердое
30,4846,7.1.2023,14,-20.58,-26.43,1019,85,-22.25,40,2.00,180,Snow,light snow,0.0,0.0,асфальт
38,4859,7.1.2023,15,-2.47,-7.32,1001,100,-2.47,0,4.00,270,Snow,light snow,0.0,0.0,грунт
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5182,13699,30.12.2023,42,-2.60,-9.60,992,75,-5.98,75,12.00,220,Snow,light shower snow,0.0,0.0,асфальт
5221,13771,30.12.2023,10,-15.53,-15.53,1007,84,-17.40,75,1.00,0,Snow,light snow,0.0,0.0,бетон
5234,13788,30.12.2023,14,-9.03,-16.03,1000,78,-11.82,75,12.07,220,Snow,light shower snow,0.0,0.0,"резиновая дорожка, мраморная плитка"
5287,13873,30.12.2023,16,-12.48,-18.49,1003,88,-13.88,100,3.00,10,Snow,light snow,0.0,0.0,смешанное
